Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Libraries

In [2]:
!pip install --pre deepchem
!pip install silence_tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 8.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for silence_tensorflow: filename=silence_tensorflow-1.2.1-py3-none-any.whl size=4475 sha256=95f5f2bbd639676088688130fdaddcd9409a9da57a3d1d734f93d9bc7ddafc78
  Stored in directory: /root/.cache/pip/wheels/7d/2c/24/e130d6102c0df56631b9db7479d9a6a53c5d97fb06b5f61b98
  Created wheel for support_developer: filename=support_developer-1.0.5-py3-none-any.whl size=5651 sha256=6806e14d33557a245bb2ac77421f56f7ad638e2b72dcee9009a60b6aeccc9140
  Stored in directory: /root/.cache/pip/wheels/b6/72/c8/3054a5897ba0713dfa7a941364d68cbd42b0755c8e2ec1c18c
S

Import Statements

In [3]:
import os
import io
import pickle
import shutil
import warnings
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
from random import seed
from random import shuffle
import matplotlib.pyplot as plt
from itertools import combinations

import deepchem as dc
from deepchem.feat.mol_graphs import ConvMol, WeaveMol
from deepchem.models.layers import GraphConv, GraphPool, GraphGather

from keras import backend as K 
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, LSTM, Dropout, BatchNormalization, Conv2D, Flatten

from scipy.stats import pearsonr
from scipy.stats import wasserstein_distance

from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.keras.layers as layers

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

#Silence Tensorflow warnings
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

#Seed Randomness
seed(13)

Load Dataset into 5 seperate folds

In [5]:
file_to_read = open("/content/drive/MyDrive/Colab Notebooks/NIST Dataset.pickle", "rb")

d = pickle.load(file_to_read)
smiles = d["smiles"]
sequences = d["sequences"]
that_index = int(np.where(smiles == "C")[0]) #This single carbon node (methane) doesn't work for some of the graphs, so we manually exclude it 
smiles = np.concatenate((smiles[:that_index], smiles[that_index+1:]))
sequences = np.concatenate((sequences[:that_index], sequences[that_index+1:]))

#Zip each data sequence
dataset = list(zip(smiles, sequences))
shuffle(dataset)

#Extract compounds that occur more than once so that repeats aren't distributed across folds
single_occurence_molecules = [x for x in dataset if list(d["smiles"]).count(x[0]) <= 1]
multiple_occurence_molecules = [x for x in dataset if x[0] not in [h[0] for h in single_occurence_molecules]]
multis = multiple_occurence_molecules

#Create folds
folds = {}
fold_size = len(single_occurence_molecules) // 5
for i in range(1, 6):
    folds[i] = single_occurence_molecules[((i - 1) * fold_size):(i * fold_size)]

#Add whatever wasn't added from signgle occurences to the end of multiple occurences
multiple_occurence_molecules += single_occurence_molecules[(5 * fold_size):]
mult_fold_size = len(multiple_occurence_molecules) // 5

#Add all these molecules across folds such that all repeat occurences always occur within the same fold
current_fold = 0
while(len(multiple_occurence_molecules) > 0):
    current_fold %= 5
    current_fold += 1
    current_molecule = multiple_occurence_molecules[0]
    while current_molecule[0] in [h[0] for h in multiple_occurence_molecules]:
        folds[current_fold].append(multiple_occurence_molecules.pop([h[0] for h in multiple_occurence_molecules].index(current_molecule[0])))

#Print the length of each fold 
for i in range(1, 6):
    print(len(folds[i]))

1501
1503
1500
1505
1496


Create test and train sets

In [6]:
#Helper Functions
def normalize(s):
    """Normalize the input series from 0->1 and return it"""
    maxval = max(s)
    scale = 1 / maxval
    if(maxval == 0):
      scale = 0
    return([j * scale for j in s])

def floor_out(x):
    """Add a floor threshold of 0.01 to reduce noise in spectra"""
    return([j if j > 0.01 else 0 for j in x])

def normal_many(x):
    """Normalize and floor in series"""
    return(np.array([floor_out(normalize(j)) for j in x]))

#Create fold sets
dataset_splits = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}
for i in range(1, 6):
  #For each i-th split, the testing set will be the i-th fold
  test = folds[i]
  train = []
  for x in range(1, 6):
    if x != i:
      train += folds[x]
  
  dataset_splits[i]["test_smiles"] = [j[0] for j in test]
  dataset_splits[i]["test_y"] = normal_many([j[1] for j in test])[:,432:] 
  dataset_splits[i]["train_smiles"] = [j[0] for j in train]
  dataset_splits[i]["train_y"] = normal_many([j[1] for j in train])[:,432:]

<ipython-input-6-39d36c2dcfca>:5: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 1 / maxval


Define Loss Functions

In [7]:
def euc_dist_keras(y_true, y_pred):
    """Euclidean distance loss function"""
    return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))

def pearson_first(y_true, y_pred):
    """Return pearson correlation for two single tensors"""
    return(pearsonr(y_true, y_pred)[0])

def wrapped_pearson_correlation(y_true, y_pred):
    y = tf.py_function(func = pearson_first, inp = [y_true, y_pred], Tout = tf.float32)
    return(y)

Run SMILES through DC Featurizer

In [8]:
featurizer = dc.feat.CircularFingerprint(radius = 2, size = 1024, chiral = False, features = False)
for i in range(1, 6):
  dataset_splits[i]["test_x"] = featurizer.featurize(dataset_splits[i]["test_smiles"])
  dataset_splits[i]["train_x"] = featurizer.featurize(dataset_splits[i]["train_smiles"])

Add Graph Featurization

In [ ]:
graph_featurizer = dc.feat.ConvMolFeaturizer()
for i in range(1, 6):
  dataset_splits[i]["test_x_graph"] = graph_featurizer.featurize(dataset_splits[i]["test_smiles"])
  dataset_splits[i]["train_x_graph"] = graph_featurizer.featurize(dataset_splits[i]["train_smiles"])
  

Graph Data Generator

In [ ]:
def data_generator(dataset, epochs = 1):
    for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs, deterministic = True, pad_batches = True)):
        multiConvMol = ConvMol.agglomerate_mols(X_b)
        inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, multiConvMol.membership]
        for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
            inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
            inputs = np.array(inputs)
            labels = [y_b]
            weights = [w_b]

    yield (inputs, labels, weights)

def clean(arr):
    #Helper Function for DC featurizers
    arr = list(map(float, arr))
    return [item for item in arr if not np.isnan(item)]

Train Each Graph Fold

Dense Models Featurizers and Training

In [ ]:
#GraphConv featurization
graph_featurizer = dc.feat.ConvMolFeaturizer()
for i in range(1, 6):
  dataset_splits[i]["test_x_graph"] = graph_featurizer.featurize(dataset_splits[i]["test_smiles"])
  dataset_splits[i]["train_x_graph"] = graph_featurizer.featurize(dataset_splits[i]["train_smiles"])

In [ ]:
#Weave featurization for MPNN
mpnn = dc.feat.WeaveFeaturizer()
for i in range(1, 6):
  dataset_splits[i]["test_x_mpnn"] = mpnn.featurize(dataset_splits[i]["test_smiles"])
  dataset_splits[i]["train_x_mpnn"] = mpnn.featurize(dataset_splits[i]["train_smiles"])
  break

MPNN

In [ ]:
#MPNN model training loop

from deepchem.models.torch_models import MPNNModel
"""
dtrain = dc.data.NumpyDataset(X = dataset_splits[i]["train_x_mpnn"], y = dataset_splits[i]["train_y"])
dtest = dc.data.NumpyDataset(X = dataset_splits[i]["test_x_mpnn"], y = dataset_splits[i]["test_y"])"""
for i in range(1, 6):
    dtrain = dc.data.NumpyDataset(X = dataset_splits[i]["train_x_mgc"], y = dataset_splits[i]["train_y"])
    dtest = dc.data.NumpyDataset(X = dataset_splits[i]["test_x_mgc"], y = dataset_splits[i]["test_y"])
    model = MPNNModel(1800, mode='regression', dropout = 0.1, batch_normalize = True, dense_layer_size=2048, batch_size = 64, n_pair_feat = 14, n_atom_feat = 75)
    model.fit(dtrain, nb_epoch = 100)
    #Collect evaluation metrics
    graph_r2s = []
    g2predictions = model.predict(dtest)
    total_r2, count = 0, 0
    for x in range(len(dataset_splits[i]["test_y"])):
        current_r2 = wrapped_pearson_correlation(normalize(g2predictions[x]), dataset_splits[i]["test_y"][x])
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        graph_r2s.append(current_r2)
        count += 1
    current_fold_loss = round(float(total_r2 / count), 5)
    print("R2 Loss for fold", i, ":", current_fold_loss)
    gr2 = clean(list(map(float, graph_r2s)))
    print("I:", i, "Mean", statistics.mean(gr2), "Median", statistics.median(gr2), "STDev", statistics.stdev(gr2))
    fold_predictions_path = path + "MPNN_" + str(i) + "_preds.pickle"
    with open(fold_predictions_path, 'wb') as handle:
        pickle.dump(g2predictions, handle)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


R2 Loss for fold 1 : 0.87099
I: 1 Mean 0.8709891633702062 Median 0.9148299098014832 STDev 0.14196605216820152
R2 Loss for fold 2 : 0.86357
I: 2 Mean 0.8635666554884184 Median 0.9149425625801086 STDev 0.1539242824899847
R2 Loss for fold 3 : 0.87453
I: 3 Mean 0.8745304910739263 Median 0.9209468364715576 STDev 0.13286753394083609


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


R2 Loss for fold 4 : 0.86081
I: 4 Mean 0.8613861383810798 Median 0.9138676822185516 STDev 0.1590294293532036
R2 Loss for fold 5 : 0.87333
I: 5 Mean 0.873326017391833 Median 0.9191482663154602 STDev 0.14286206732989026


AttentiveFP 

In [ ]:
#AttentiveFP model training loop
import deepchem as dc
from deepchem.models import AttentiveFPModel
for i in range(1, 6):
    dtrain = dc.data.NumpyDataset(X = dataset_splits[i]["train_x_mgc"], y = dataset_splits[i]["train_y"])
    dtest = dc.data.NumpyDataset(X = dataset_splits[i]["test_x_mgc"], y = dataset_splits[i]["test_y"])
    fpmodel = AttentiveFPModel(n_tasks = 1800, mode='regression', dropout = 0.1, batch_normalize = True, dense_layer_size=2048, batch_size = 64, learning_rate = 0.001, optimizer = "Adam", activation_fns = "p")

    fpmodel.fit(dtrain, nb_epoch = 100)
    fppredictions = fpmodel.predict(dtest)

    graph_r2s = []
    total_r2, count = 0, 0
    for x in range(len(dataset_splits[i]["test_y"])):
        current_r2 = wrapped_pearson_correlation(normalize(fppredictions[x]), dataset_splits[i]["test_y"][x])
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        graph_r2s.append(current_r2)
        count += 1
    current_fold_loss = round(float(total_r2 / count), 5)
    print("R2 Loss for fold", i, ":", current_fold_loss)
    gr2 = clean(list(map(float, graph_r2s)))
    print("I:", i, "Mean", statistics.mean(gr2), "Median", statistics.median(gr2), "STDev", statistics.stdev(gr2))#expirement
    fold_predictions_path = path + "AFP_" + str(i) + "_preds.pickle"
    with open(fold_predictions_path, 'wb') as handle:
        pickle.dump(fppredictions, handle)    

R2 Loss for fold 1 : 0.89004
I: 1 Mean 0.8900437510084066 Median 0.9359551072120667 STDev 0.13420706069836466
R2 Loss for fold 2 : 0.88568
I: 2 Mean 0.8856813292343142 Median 0.9355883002281189 STDev 0.13951395890818105
R2 Loss for fold 3 : 0.89097
I: 3 Mean 0.8909696869750817 Median 0.9375252425670624 STDev 0.12860900264472283


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


R2 Loss for fold 4 : 0.88039
I: 4 Mean 0.8809766138871596 Median 0.9358482956886292 STDev 0.1528139015404319
R2 Loss for fold 5 : 0.88751
I: 5 Mean 0.8875074465734197 Median 0.9358925521373749 STDev 0.14031811902949104


GAT Model

In [ ]:
#GAT model training loop
for i in range(1, 6):
    dtrain = dc.data.NumpyDataset(X = dataset_splits[i]["train_x_mgc"], y = dataset_splits[i]["train_y"])
    dtest = dc.data.NumpyDataset(X = dataset_splits[i]["test_x_mgc"], y = dataset_splits[i]["test_y"])
    #64, 64 imrpobed it
    model = dc.models.GATModel(1800, mode='regression', dropout = 0.1, graph_attention_layers = [64, 64], batch_normalize = True, dense_layer_size=2048, batch_size = 64, learning_rate = 0.001)
    model.fit(dtrain, nb_epoch = 100)
    gatpredictions = model.predict(dtest)

    graph_r2s = []
    total_r2, count = 0, 0
    for x in range(len(dataset_splits[i]["test_y"])):
        current_r2 = wrapped_pearson_correlation(normalize(gatpredictions[x]), dataset_splits[i]["test_y"][x])
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        graph_r2s.append(current_r2)
        count += 1
    current_fold_loss = round(float(total_r2 / count), 5)
    print("R2 Loss for fold", i, ":", current_fold_loss)
    gr2 = clean(list(map(float, graph_r2s)))
    print("I:", i, "Mean", statistics.mean(gr2), "Median", statistics.median(gr2), "STDev", statistics.stdev(gr2))#expirement
    
    fold_predictions_path = path + "GAT_" + str(i) + "_preds.pickle"
    with open(fold_predictions_path, 'wb') as handle:
        pickle.dump(gatpredictions, handle) 

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
R2 Loss for fold 1 : 0.80144
I: 1 Mean 0.8014362164334686 Median 0.8348127603530884 STDev 0.15754606698103107
R2 Loss for fold 2 : 0.80858
I: 2 Mean 0.8085786684361086 Median 0.8482959270477295 STDev 0.15179206103721388
R2 Loss for fold 3 : 0.80534
I: 3 Mean 0.8053351333787044 Median 0.8364363312721252 STDev 0.14961929769952964


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


R2 Loss for fold 4 : 0.80272
I: 4 Mean 0.8032512624049559 Median 0.8420479595661163 STDev 0.16012092895721208
R2 Loss for fold 5 : 0.80157
I: 5 Mean 0.801570797378302 Median 0.831894725561142 STDev 0.1479842578919792


MorganFP/DNN Model

In [ ]:
#MorganFP model using a dense layer as output
for i in range(1, 6):
    fpmodel = Sequential()
    fpmodel.add(Dense(4096, input_dim = 1024))
    fpmodel.add(BatchNormalization())
    fpmodel.add(Dropout(0.1))
    fpmodel.add(Dense(2048, activation = "relu"))
    fpmodel.add(BatchNormalization())
    fpmodel.add(Dropout(0.1))
    fpmodel.add(Dense(1024, activation = "relu"))

    fpmodel.add(Dense(1800, activation = "sigmoid"))

    #opt = Adam(learning_rate = 0.001)

    fpmodel.compile(loss = euc_dist_keras, optimizer = "Adam")
    fpmodel.fit(dataset_splits[i]["train_x"], dataset_splits[i]["train_y"], batch_size = 64, epochs = 100, verbose = 0)
    #Collect evaluation metrics
    morganpredictions = fpmodel.predict(dataset_splits[i]["test_x"])
    total_r2, count = 0, 0
    totalp = 0
    fp_r2s = []
    for x in range(len(morganpredictions)):
        current_r2 = wrapped_pearson_correlation(normalize(morganpredictions[x]), dataset_splits[i]["test_y"][x])
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        fp_r2s.append(current_r2)
        count += 1

    current_fold_loss = round(float(total_r2 / count), 5)
    print("R2 Loss for fold", i, ":", current_fold_loss)
    cleanfpr2s = clean(list(map(float, fp_r2s)))
    print("I", i, statistics.mean(cleanfpr2s), statistics.median(cleanfpr2s), statistics.stdev(cleanfpr2s))
    fold_predictions_path = path + "MFP_" + str(i) + "_preds.pickle"
    with open(fold_predictions_path, 'wb') as handle:
        pickle.dump(morganpredictions, handle)    

R2 Loss for fold 1 : 0.86765
I 1 0.8676516930898435 0.9199491143226624 0.15140122721398225
R2 Loss for fold 2 : 0.8645
I 2 0.8644993320399931 0.9224254488945007 0.1577399099835786
R2 Loss for fold 3 : 0.86978
I 3 0.8697819181475789 0.92546346783638 0.147056175220565


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


R2 Loss for fold 4 : 0.85942
I 4 0.8599970634491678 0.9204980432987213 0.16062992959480313
R2 Loss for fold 5 : 0.86863
I 5 0.8686325302681224 0.9234545826911926 0.1527444680999287


Load up pickle files of saved predictions

In [ ]:
path = "/content/drive/MyDrive/Miscellaneous Code/SpectraPredictions/"
all_predictions = {'GC': [], 'MPNN': [], 'AFP': [], 'GAT': [], 'MFP': []}
for model_string in ('GC_', 'MPNN_', 'AFP_', 'GAT_', 'MFP_'):
    for i in range(1, 6):
        fold_predictions_path = path + model_string + str(i) + "_preds.pickle"
        with open(fold_predictions_path, 'rb') as handle:
            all_predictions[model_string[:-1]].extend(pickle.load(handle))
len(all_predictions['GC'])

7505

In [ ]:
#Normal average manually added up from model predictions
total_r2, count = 0, 0
all_r2 = {'GC': [], 'MPNN': [], 'AFP': [], 'GAT': [], 'MFP': []}
for model_string in ('GC', 'MPNN', 'AFP', 'GAT', 'MFP'):
    for x in range(7505):
        current_r2 = wrapped_pearson_correlation(normalize(all_predictions[model_string][x]), normalize(all_labels[x]))
        all_r2[model_string].append(current_r2)
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        count += 1
    print(model_string, round(float(total_r2 / count), 5)) 

<ipython-input-7-c52eb143fa4e>:75: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 1 / maxval
/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


GC 0.79164
MPNN 0.83014
AFP 0.84906
GAT 0.83778
MFP 0.84342


In [ ]:
#This block does both flooring and gaus smoothing at the right value 
def floor_pred(pred):
    floored_pred = [absorbance if absorbance >= 0 else 0 for absorbance in pred]
    return floored_pred
import scipy
total_r2, count = 0, 0
all_r2_flat_smooth = {'GC': [], 'MPNN': [], 'AFP': [], 'GAT': [], 'MFP': []}
#all_r2_flat_smooth = {'AFP': []}
for model_string in all_r2_flat_smooth.keys():
    for x in range(7505):
        current_r2 = wrapped_pearson_correlation(scipy.ndimage.gaussian_filter1d(floor_pred(normalize(all_predictions[model_string][x])), 3), normalize(all_labels[x]))
        all_r2_flat_smooth[model_string].append(current_r2)
        total_r2 += 0 if np.isnan(current_r2) else current_r2 
        count += 1
    print(model_string, round(float(total_r2 / count), 5)) 

<ipython-input-7-c52eb143fa4e>:75: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 1 / maxval
/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


GC 0.80137
MPNN 0.83668
AFP 0.85424
GAT 0.84324
MFP 0.84791
